# Homework (deadline 25.11.2022 11:59:59)
Write solutions for the homework exercises in this notebook. Once the work is done download the notebook file (`File > Download .ipynb`) rename it properly so it follows a template `HW1_<SURNAME>_<NAME>.ipynb` and send the file to me. **Please also attach all files you will be asked to produce in this homework**. My email address is as follows: 

* <m.biesaga@uw.edu.pl>

Remember that you can contact me via email if you have any problems. Moreover, you can also visit me in the ISS on the fourth floor (room 415). Usually, I am there from 11ish but please let me know in advance if you are coming because I might be busy. 


## Task 1 (5 points)

Connect to Twitter filtered-stream during a live event of your choice (the easiest would be a sporting event because they usually have a specific hashtag) and collect 100 tweets that are not in English and are not retweets. Among the tweets find a user with the most followers and download 300 tweets from their timeline. Save the tweets in a JSON line file, however, please write out only the following fields:

* `retweet count`
* `like count`
* `quote count`
* `reply count`
* `id`
* `context_annotations`
* `lang`
* `text`
* `created_at`
* `screen_name`

**HINT**: The JSON line file should contain a list of dictionaries looking more or less like this:

```python
{
  'retweet_count' : 2,
  'like_count' : 15,
  'quote_count' : 0,
  'reply_count' : 21,
  'id' : 13290821347,
  'context_annotations' : 
    [
      {
        'entity': {'id': '854692455005921281',
        'name': 'Science',
        'description': 'Science'}
      }
    ]

  'lang' : 'pl',
  'text' : 'POLSKA GUROM! #Ortografiada'
  'created_at' : '2022-11-11 11:11:11'
  'username' : "manolito_gafotas"
}

```

In [ ]:
## If you are using Google Colab otherwise comment it
!pip install twarc

## Here paste your Bearer Token but please remove it before sending me the
## solution
BEARER_TOKEN = ''

## Load module
from twarc import Twarc2, expansions
import json

## Connect to Twitter API
client = Twarc2(bearer_token=BEARER_TOKEN)

In [ ]:
## Check the existing rules. 
existing_rules = client.get_stream_rules()
print(existing_rules)

## Check whether there are any rules.
if 'data' in existing_rules and len(existing_rules['data']) > 0:
	## Return a list of rules ids
    rule_ids = [ rule['id'] for rule in existing_rules['data'] ]
	## Remove existing rules
    client.delete_stream_rule_ids(rule_ids)

In [ ]:
## Add new rules
new_rules = [
    { "value": "Qatar2022 -lang:en -is:retweet", "tag": "Qatar2022 tweets not in English" }
]
added_rules = client.add_stream_rules(rules=new_rules)

In [ ]:
## Create an empty list
list_tweets = []

## Connect to the filtered stream
for count, result in enumerate(client.stream()):
    ## Use expansions.flatten() to merge the data about a single
    ## tweet
    tweet = expansions.flatten(result)
    ## Append the tweet to the list
    list_tweets.append(tweet[0])
    ## Break the look after reaching 102 tweets
    if count > 98:
        break

## Delete the rules once you have collected the desired number of Tweets
rule_ids = [rule['id'] for rule in added_rules['data']]
client.delete_stream_rule_ids(rule_ids)

In [ ]:
## Define variables that will be populated with
## the data about the most followed user
the_most_followed_user = ''
followers_count = 0

## Iterate over all gathered tweets
for item in list_tweets:
    ## Check if the author has more followers than the previous one
    if item['author']['public_metrics']['followers_count'] > followers_count:
        ## If yes update the followers count
        followers_count = item['author']['public_metrics']['followers_count']
        ## If yes update the user name
        the_most_followed_user = item['author']['username']

In [ ]:
## Connect to the timeline of Nature 
user_timeline = client.timeline(user=the_most_followed_user, max_results = 100)

## Open connection to the file
with open('timeline.jl', 'w') as file:
  ## Twitter API will return the data paginated. It means
  ## that it will be grouped in batches of 100 tweets.
  ## Therefore, we first iterate over pages (batches of
  ## tweets).
  for count, page in enumerate(user_timeline):
      ## Go over only first 3 batches of tweets
      if count == 3:
        break
      ## Use expansions.flatten() to merge information 
      ## about tweets
      result = expansions.flatten(page)
      ## Iterate over this 100 tweets
      for tweet in result:
        ## Create an empty dictionary
        output = {}	
        ## Assign various metrics to relevant keys in the output dictionary
        output['retweet count'] = tweet['public_metrics']['retweet_count']
        output['like count'] = tweet['public_metrics']['like_count']
        output['quote count'] = tweet['public_metrics']['quote_count']
        output['reply count'] = tweet['public_metrics']['reply_count']
        output['id'] = tweet['id']
        output['lang'] = tweet['lang']
        output['text'] = tweet['text']
        output['created_at'] = tweet['created_at']
        output['screen_name'] = tweet['author']['username']
        ## Write out the resutls
        file.write(json.dumps(output) + '\n')
  